<a href="https://colab.research.google.com/github/rob-viana/Desafio-Alura---Gemini-AI/blob/main/Assistente_RH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação do SDK Google

In [ ]:
!pip install -q -U google-generativeai

In [6]:
#configuraçãoes e importações
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
minha_chave = userdata.get('SECRET_KEY')
genai.configure(api_key=minha_chave)

Listando os modelos disponiveis

In [ ]:
for mo in genai.list_models():
  if "embedContet" in mo.supported_generation_methods:
    print(mo.name)

Configurações de temperatura de respostas:

In [ ]:
configuracoes_basicas = {
    "candidate_count":1,
    "temperature":0.5,
}

Parametrizando as configuraçãoes de segurança.
Importante para o caso de necessidade de alteração.

In [ ]:
configuracoes_seguranca = {
    "Harassment": "Block_none",
    "Hate": "Block_none",
    "Sexual": "Block_none",
    "Dangerous": "Block_None",
}

In [ ]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=configuracoes_basicas,
                              safety_settings=configuracoes_seguranca)

In [ ]:
for model in genai.list_models():
  if "embedContent" in model.supported_generation_methods:
    print(model.name)

models/embedding-001
models/text-embedding-004


In [4]:
# Exemplo de embedding
title = "Vaga para Gerente Comercial"
sample_text = ("Título: Vaga para Gerente Comercial"
                        "\n"
                        "Procuramos Gerente de Vendas para desenvolver estratégias, alcançar metas e garantir a satisfação do cliente na Academia da Face. Esta é uma oportunidade única para quem deseja fazer parte de uma empresa inovadora e em crescimento no setor de estética.Experiência comprovada em vendas. Preferencialmente, conhecimento no setor de estética e bem-estar. Habilidades excepcionais de comunicação e negociação. Excelente organização e gerenciamento de tempo. Foco na satisfação e experiência do cliente.")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embeddings)


{'embedding': [0.06305395, -0.01364274, -0.010241299, 0.008322683, 0.023045942, 0.023640364, 0.013717146, 0.027174179, -0.0124367485, 0.0685895, -0.015456245, 0.016340101, 0.012679756, -0.011257882, 0.03884935, -0.018105188, -0.03918156, -0.028344762, -0.039546326, -0.022913149, 0.02030854, 0.002989264, -0.010270103, -0.035034247, -0.010353216, -0.0142493695, 0.023864685, -0.017607009, -0.009771889, 0.029590053, -0.0745057, 0.042787325, -0.050783493, -0.0025974778, -0.004975191, -0.07211683, -0.03716371, -0.023269465, -0.026879862, 0.0037419104, 0.013770794, -0.017478043, -0.014780433, -0.037125517, 0.007034199, -0.00048605926, 0.008418333, 0.052788373, 0.024450082, -0.10183931, 0.0063431533, 0.010482046, 0.032871306, 0.021232273, -0.0025924516, -0.05677812, 0.046659432, -0.008769476, -0.036160607, -0.010105197, 0.022317447, 0.014960988, 0.032723438, 0.03610762, -0.0023161368, -0.0038261816, -0.061902083, 0.015106693, 0.023525352, 0.06800396, -0.0100297555, -0.037644595, 0.03878087, 0.

Função para importação de currículos dos candidatos para comparação com o modelo desejado!

In [15]:
import pandas as pd
!pip install -q ipywidgets
from google.colab import drive

In [23]:
from ipywidgets import widgets

# Criar widget para upload de arquivo
upload_widget = widgets.FileUpload(
    multiple=False,  # Permitir apenas um arquivo por upload
    accept='.pdf'  # Limitar o tipo de arquivo para PDF
)

In [47]:

# Definir uma função para lidar com o arquivo enviado
def handle_uploaded_file(change):
    uploaded_file = change['new']  # Obter o arquivo enviado
    if uploaded_file:  # Verificar se o arquivo foi enviado
        # Obter o nome do arquivo
        filename = uploaded_file["name"]
        # Obter o conteúdo do arquivo
        file_content = uploaded_file['contents']

        # Salvar o arquivo PDF em uma pasta específica
        with open('https://drive.google.com/drive/folders/1VofQlR3bTbYzcc5eG-boDvfYInqwPP1h' + filename, 'wb') as f:
            f.write(file_content)
        print(f"Arquivo {filename} salvo com sucesso!")

# Conectar a função de manipulação ao evento de upload
upload_widget.observe(handle_uploaded_file, names='value')

# Exibir o prompt de upload
display(upload_widget)

FileUpload(value={'Currículo Advogada.pdf': {'metadata': {'name': 'Currículo Advogada.pdf', 'type': 'applicati…

Exibindo resultados

In [45]:

def compare_file_content_to_embedding(filename, file_content, title, sample_text):
    # Convert file content to text
    file_text = file_content.decode('utf-8')

    # Combine title, sample text, and file content into a single text
    combined_text = title + '\n\n' + sample_text + '\n\n' + file_text

    # Create a DataFrame from the combined text
    df = pd.DataFrame({'text': [combined_text]})

    # Calculate the embedding for the combined text
    # (Replace 'embedding_model' with your chosen embedding model)
    embedding = embedding_model.predict(df['text'])

    # Compare the embedding of the uploaded file to the embedding of the sample text
    similarity = cosine_similarity(embedding, sample_embedding)

    # Print the similarity score
    print(f"Similarity score: {similarity}")

    # You can also perform additional actions based on the similarity score
    # (e.g., classify the file as relevant or not)


In [46]:

def compare_file_content_to_embedding(filename, file_content, title, sample_text):
    # Convertendo file content para texto
    file_text = file_content.decode('utf-8')

    # Combine title, sample text, e file content into a single text
    combined_text = title + '\n\n' + sample_text + '\n\n' + file_text

    # Create a DataFrame from the combined text
    df = pd.DataFrame({'text': [combined_text]})

    # Calculate the embedding for the combined text
    # (Replace 'embedding_model' with your chosen embedding model)
    embedding = embedding_model.predict(df['text'])

    # Compare the embedding of the uploaded file to the embedding of the sample text
    similarity = cosine_similarity(embedding, sample_embedding)

    # Print the similarity score
    print(f"Similarity score: {similarity}")

    # You can also perform additional actions based on the similarity score
    # (e.g., classify the file as relevant or not)
